# Cálculo do VaR para uma carteira de ações

In [2]:
!pip install yfinance
!pip install yahoofinancials
import yfinance as yf
import pandas as pd
import numpy as np
import math as mt

Após a instalação do API yfinance, definimos uma variável para o Ticker de cada ação a fim de facilitar a utilização desta informação nos trechos seguintes do programa.

Em seguida, fora extraido os dados históricos dos papéis escolhidos para um período desejado e podemos observar que nesta parte já utilizamos a variável definida para o Ticker do papél.

In [3]:
CCR = yf.Ticker('CCRO3.SA')
BB = yf.Ticker('BBAS3.SA')
JBS = yf.Ticker('JBSS3.SA')
TIM = yf.Ticker('TIMS3.SA')

CCR_data = CCR.history( start ='2021-06-30', end = '2022-07-01')
BB_data = BB.history(  start ='2021-06-30', end = '2022-07-01')
JBS_data = JBS.history( start ='2021-06-30', end = '2022-07-01')
TIM_data = TIM.history( start ='2021-06-30', end = '2022-07-01')


Neste trecho, já realizamos alguns cálculos e tratamentos nos dados obtidos para realizar os respectivos cálculos do VaR. 

Inicialmente, fora realizado o cálculo da Rentabilidade diária das ações, no período considerado, dividindo o preço da ação de um dia pelo preço dela no dia anterior, menos 1.

In [4]:
CCR_data['RetornoC'] = (CCR_data['Close']/CCR_data['Close'].shift(-1))-1
BB_data['RetornoB'] = (BB_data['Close']/BB_data['Close'].shift(-1))-1
JBS_data['RetornoJ'] = (JBS_data['Close']/JBS_data['Close'].shift(-1))-1
TIM_data['RetornoT'] = (TIM_data['Close']/TIM_data['Close'].shift(-1))-1


Em seguida, realizei alguns tratamentos com o objetivo de reduzir as informações presentes nas tabelas extraídas, assim como, criar outras variáveis que serão utilizadas ao longo do programa.

- Primeiramente, retirei algumas colunas presentes nos históricos extraídos e que não serão utilizadas neste estudo.


- Como no primeiro dia da série, não temos o dia anterior para calcular o retorno da ação, com isso, o campo de retorno ficou com valor NaN, o  que poderia comprometer outros cálculos, por isso, esses campos foram preenchidos com 0 para não interferir no desenvolvimento da simulação.


- Por último, defini uma coluna para índices destas tabelas, visando facilitar na manipulação das mesmas, pois antes estavam como índices as datas dos períodos extraídos, o que prejudicaria em alguns cálculos.
  
  
  

In [5]:
CCR = CCR_data.drop(columns=['Open','High','Low','Volume','Dividends','Stock Splits'])
BB = BB_data.drop(columns=['Open','High','Low','Volume','Dividends','Stock Splits'])
JBS = JBS_data.drop(columns=['Open','High','Low','Volume','Dividends','Stock Splits'])
TIM = TIM_data.drop(columns=['Open','High','Low','Volume','Dividends','Stock Splits'])


CCR['RetornoC'] = CCR['RetornoC'].fillna(0)
BB['RetornoB'] = BB['RetornoB'].fillna(0)
JBS['RetornoJ'] = JBS['RetornoJ'].fillna(0)
TIM['RetornoT'] = TIM['RetornoT'].fillna(0)

CCR.index = range(250)
BB.index = range(250)
JBS.index = range(250)
TIM.index = range(250)

In [6]:
tamanho = len(CCR.index)
tamanho

250

Tendo em vista uma melhor manipulação e utilização dos retornos calculados, transformei em séries as colunas referentes a eles nas tabelas extraídas.

In [7]:
CCR_S = pd.Series(CCR['RetornoC'], name = 'CCR')
BB_S = pd.Series(BB['RetornoB'], name = 'BB')
JBS_S = pd.Series(JBS['RetornoJ'], name = 'JBS')
TIM_S = pd.Series(TIM['RetornoT'], name = 'TIM')

Dado que estou considerando um nível de confiança de 95%, precisamos calcular para cada papel, o retorno que abaixo dele estão 
5% dos piores retornos observados, sendo a partir deste valor que calcularemos o VaR. 

Multiplicação por -1, realizada para facilitar o cálculo do VaR nos passos seguintes.

In [24]:
CCR_P = np.percentile(CCR_S, 5)*(-1)
BB_P = np.percentile(BB_S, 5)*(-1)
JBS_P = np.percentile(JBS_S, 5)*(-1)
TIM_P = np.percentile(TIM_S, 5)*(-1)

In [ ]:
Desta forma, calculado o Percentil que representa a perda máxima em 95% das vezes, podemos agora definir a perda máxima em um
dia, em termos monetários.

Primeiro, obtemos o volume de cada papel no dia que estamos calculando o VaR que neste caso é '01-07-2022'.
Cada papel apresenta uma quantidade de 1000 unidades.

In [11]:
Qt = 1000

In [12]:
volumeC = CCR.iloc[20,0] * Qt
volumeB = BB.iloc[20,0] * Qt
volumeJ = JBS.iloc[20,0] * Qt
volumeT = TIM.iloc[20,0] * Qt

In [25]:
varC = volumeC * CCR_P
varB = volumeB * BB_P
varJ = volumeJ * JBS_P
varT = volumeT * TIM_P

In [26]:
var = varC + varB + varJ + varT
var

2559.0865104375234

In [ ]:
Calculado o VaR, precisamos agora fazer o Back Testing do modelo utilizado, com o objetivo de validá-lo. Isto é realizado 
a partir do levantamento da quantidade de dias que excederam a perda estimada no VaR, e com isso, espera-se que estejam em
uma porcentagem igual ou aproximada ao nível de significância, neste estudo é de 5%.

In [ ]:
Desta forma, de início, calculei os volumes diários dos papéis que compõem a carteira, utilizando a quantidade definida e as
cotações diárias extraídas. E, em seguida, obtemos a posição total da carteira por dia, somando as posições desses papéis.

In [27]:
CCR_V = pd.Series(CCR['Close']*Qt, name = 'CCR_V')
BB_V = pd.Series(BB['Close']*Qt, name = 'BB_V')
JBS_V = pd.Series(JBS['Close']*Qt, name = 'JBS_V')
TIM_V = pd.Series(TIM['Close']*Qt, name = 'TIM_V')


Cart = pd.Series(CCR_V + BB_V + JBS_V + TIM_V, name = 'Cart')
Cart


0      80083.174706
1      79720.569611
2      80398.632050
3      80084.711075
4      78632.308960
           ...     
245    90789.999962
246    91620.001793
247    91839.999199
248    90950.000763
249    90260.001183
Name: Cart, Length: 250, dtype: float64

In [ ]:
Encontradas as posições diárias da carteira, podemos calcular o retorno diário desta para que possamos compará-los com o VaR.

In [28]:
Btest = pd.DataFrame(Cart)
Btest['Retorn'] = (Btest['Cart'].shift(1) - Btest['Cart'])
Btest['Retorn'] = Btest['Retorn'].fillna(0)
Btest

,Cart,Retorn
0,80083.174706,0.000000
1,79720.569611,362.605095
2,80398.632050,-678.062439
3,80084.711075,313.920975
4,78632.308960,1452.402115
...,...,...
245,90789.999962,-351.634979
246,91620.001793,-830.001831
247,91839.999199,-219.997406
248,90950.000763,889.998436


Para comparação dos retornos diários com o VaR foi preciso utilizar uma estrutura de Looping For, onde inicializamos uma variável com receberá o valor do VaR (a) e outra variável inicializada como zero e que ao longo do Loop receberá a contagem dos dias que a perda da carteira foi superior ao VaR (b).

In [29]:
a = var

Btest['Var'] = a
a

2559.0865104375234

In [31]:
Carteira = Btest['Retorn'].tolist()
b=0
for i in Carteira:
    if(i>a):
        b+=1

In [ ]:
Calculados os dias em que a perda superou o VaR, fazemos agora uma proporção desta contagem com os dias totais observados na 
amostra e, com isso, encontramos que em aproximadamente 4% das observações a perda excedeu o VaR estimado.

In [32]:
Fora_var = (b/len(Btest.index))
Fora_var


0.04

Logo, a partir dos cálculos realizados podemos observar que o modelo utilizado está dentro dos limites considerados, podendo ser utilizado para tomadas de decisões referentes a reservas de capitais, por exemplo, ou mesmo outras estratégias financeiras. 
No entanto, este modelo de certa forma ainda pode apresentar melhorias dado que podemos considerar uma superestimação deste, pois as observações que excedem o VaR ainda estão abaixo do nível de significância definido o que pode levar a uma alocação ineficiente de capital, fazendo com que reservas sejam realizadas com valores acima do necessário.

Como dito no início de estudo, um dos principais objetivos de sua realização foi a consolidação do aprendizado em programação na liguagem Python, bem como sua aplicação em um campo de conhecimento na área em que atuo profissionalmente. Ainda assim, seria preciso um maior número de informações para tomada de decisões mais assertivas ou até mesmo a utilização de outros modelos para cálculo do VaR, dado que o modelo de Simulação Histórica possui alguns aspectos que deixam seus resultados menos rigorosos em comparação com outros modelos.